# **1. Kết nối với google drive**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NLP 4+1/Cuối kỳ/')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/NLP 4+1/Cuối kỳ'

# **2. Install và import các thư viện cần thiết**

In [ ]:
!pip install transformers --quiet
!pip install pyvi --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.8 MB/s eta 0:00:00


In [ ]:
import torch
import time
import pandas as pd
from pyvi import ViTokenizer
from tqdm.notebook import tqdm

from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

from transformers import GPT2LMHeadModel, GPT2Model
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer
from transformers import TextDataset, GPT2Tokenizer, AutoTokenizer

# **3. Set các biến cục bộ**

In [ ]:
import pickle

# saving
def save_file(data, path):
    with open(path + '.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
def load_file(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
EPOCHS = 100
BATCH_SIZE = 32
path_data = './dataset/'                # đường dẫn data
path_pre_trained = './pre-trained/GPT/'  # đường dẫn chứa các model đã train

In [ ]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('danghuy1999/gpt2-viwiki')
model = GPT2LMHeadModel.from_pretrained('danghuy1999/gpt2-viwiki')
# save
save_file(model, path_pre_trained + 'GPT2LMHeadModel')

Some weights of the model checkpoint at danghuy1999/gpt2-viwiki were not used when initializing GPT2LMHeadModel: ['multiple_choice_head.summary.bias', 'multiple_choice_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
print("Number of parameters in model: ", model.num_parameters())

Number of parameters in model:  124439808


In [ ]:
print("Vocabulary size: ", tokenizer.vocab_size)
print("Maximum model input length: ", tokenizer.max_model_input_sizes['gpt2'])

Vocabulary size:  50257
Maximum model input length:  1024


In [ ]:
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                              "bos_token": "<s>",
                              "eos_token": "</s>"})
tokenizer.add_tokens(["<bot>:"])

# save
save_file(tokenizer, path_pre_trained + 'tokenizer')

In [ ]:
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# **4. Xây dựng class để tạo dataset cho quá trình huấn luyện**

In [ ]:
class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = pd.read_csv(path, usecols = [0,1])

        self.X = []
        for q,a in self.data.values:
            self.X.append('<s> '+ q +" <bot>: "+ a +' </s>')
        
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, max_length=32, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
chatData = ChatData(path_data + 'train.csv', tokenizer)
chatData =  DataLoader(chatData, batch_size=BATCH_SIZE)

<s> có thưởng tết không <bot>: có </s>


# **5. Xây dựng hàm để traning và trả lời câu hỏi từ model**

In [ ]:
def train(chatData, model, optim, epochs):
    loss_history = []
    for i in range(epochs):
        total_loss = 0
        print('======== Epoch {:} / {:} ========'.format(i + 1, epochs))
        for X, a in tqdm(chatData):
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            total_loss += loss.item()
            loss.backward()
            optim.step()
        avg_train_loss = total_loss / len(chatData)  
        loss_history.append(avg_train_loss)
        torch.save(model.state_dict(), path_pre_trained + "model_GPT.pt")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print(bot_give_answer("bạn học ngành gì ?"))
    return loss_history
    
def bot_give_answer(inp):
    inp = '<s> '+ inp +" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    mask = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=mask, max_length=32, pad_token_id=0, eos_token_id=50256)
    output = tokenizer.decode(output[0])
    return output

In [ ]:
# training
model.train()
optim = Adam(model.parameters(), lr=1e-3)

t0 = time.time()
loss_history = train(chatData, model, optim, epochs=EPOCHS)
time_train = time.time() - t0

======== Epoch 1 / 100 ========


  0%|          | 0/336 [00:00<?, ?it/s]

  Average training loss: 2.99
<s> bạn học ngành gì? <bot>: mình học ngành gì </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 2 / 100 ========


  0%|          | 0/336 [00:00<?, ?it/s]

  Average training loss: 2.00
<s> bạn học ngành gì? <bot>: mình học ngành khoa_học máy_tính </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 3 / 100 ========


...

  Average training loss: 0.27
<s> bạn học ngành gì? <bot>: tôi học cntt </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 99 / 100 ========


  0%|          | 0/336 [00:00<?, ?it/s]

  Average training loss: 0.27
<s> bạn học ngành gì? <bot>: bản_xuất </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 100 / 100 ========


  0%|          | 0/336 [00:00<?, ?it/s]

  Average training loss: 0.27
<s> bạn học ngành gì? <bot>: tôi học ngành kỹ_thuật phần_mềm </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [ ]:
val_compare = {
    'loss_history': loss_history,
    'time_train': time_train
}

# save
save_file(val_compare, path_pre_trained + 'val_compare')